<div>
<img src="https://i.ibb.co/v3CvVz9/udd-short.png" width="150"/>
    <br>
    <strong>Universidad del Desarrollo</strong><br>
    <em>Magíster en Data Science</em><br>
    <em>Profesora: Maria Paz Raveau</em><br>
    <em>Asignatura: Procesamiento de Lenguaje Natural</em><br>

</div>

# **Tarea 1**

*Fecha de Entrega: Domingo 20, Julio 2025.*

**Nombre Estudiante**: Victor Saldivia Vera

### **01. Intrucciones y Enunciado**

Suponga que ud. es contactado por alguna agencia gubernamental que quiere saber qué dijeron los participantes de los cabildos de 2016. Ud. debe entonces, basándose en los textos de fundamento, responder a este requerimiento.  Use para ello la base de datos que vimos en clases, de los cabildos provinciales. 

- Escoja alguna de los conceptos que vienen en la base de datos (evite las que no tengan muchas entradas). Hemos visto cómo generar WordClouds con tokens y con bigramas. *¿Es posible valerse de las etiquetas gramaticales para mejorar el resultado?* 
- Pruebe haciendo un WordCloud con tokens/sustantivos,  y otro con  bigramas/sustantivo-adjetivo. Recuerde pre-procesar correctamente.
- Con estos mismo bigramas/sustantivo-adjetivo, haga una red de bigramas.
- Ayúdese con frases de ejemplo, y de los WordClouds, para escribir una descripción del concepto.


### **02. Descripción de la Data**


In [1]:
import pandas as pd
df = pd.read_excel('data/resultadocabildoprovincial.xlsx')
print(f'Dimensión: {df.shape}'), 
print(f'Nombre de Columnas: {df.columns}')

Dimensión: (30414, 12)
Nombre de Columnas: Index(['region_nombre', 'provincia_nombre', 'comuna_nombre', 'cabildo_id',
       'cabildo_nombre', 'cabildo_direccion', 'num_mesa', 'pregunta_glosa',
       'concepto_nombre', 'concepto_otro', 'tipo-eleccion', 'fundamento'],
      dtype='object')


### **03. Librerías a Utilizar**


In [2]:
import pandas as pd
import numpy as np
import re
import unicodedata
from collections import Counter

# Librerías de Procesamiento de Texto
import nltk
from nltk.corpus import stopwords
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
import stanza

# Visualización
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import networkx as nx

# La librería de warnings se utiliza para manejar las advertencias
import warnings
warnings.filterwarnings('ignore')

# Paquete muestra Barra de Progreso
from tqdm import tqdm
tqdm.pandas()

c:\Users\Victo\anaconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Descargas (Nota: Ejecutar una sola vez en entorno local)
# nltk.download('punkt')
# nltk.download('stopwords')
# stanza.download('es')

#### Configuración de Stanza para el PLN

In [3]:
nlp = stanza.Pipeline('es', processors='tokenize,pos,lemma', use_gpu=False)  # Cambiar a True si hay GPU


2025-07-20 14:12:21 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-07-20 14:12:22 INFO: Downloaded file to C:\Users\Victo\stanza_resources\resources.json
2025-07-20 14:12:22 WARNING: Language es package default expects mwt, which has been added
2025-07-20 14:12:22 INFO: Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |

2025-07-20 14:12:22 INFO: Using device: cpu
2025-07-20 14:12:22 INFO: Loading: tokenize
2025-07-20 14:12:26 INFO: Loading: mwt
2025-07-20 14:12:26 INFO: Loading: pos
2025-07-20 14:12:29 INFO: Loading: lemma
2025-07-20 14:12:31 INFO: Done loading processors!


### **04. Carga y Exploración Inicial**


In [4]:
df.head()

,region_nombre,provincia_nombre,comuna_nombre,cabildo_id,cabildo_nombre,cabildo_direccion,num_mesa,pregunta_glosa,concepto_nombre,concepto_otro,tipo-eleccion,fundamento
0,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Autonomía/libertad,NaN,A,sin fundamento
1,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Sin fundamento
2,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Descentralización,NaN,A,La descentralizacion de ser autonoma por ser z...
3,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Equidad de género,NaN,A,Sin fundamento
4,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Respeto/conservación de la naturaleza o medio ...,NaN,A,Sin fundamento


#### Conteo de nulos en columnas de texto


In [5]:
texto_col = "fundamento"
print(df[texto_col].isna().sum())

0


#### Estadísticas Descriptivas sobre el texto


In [6]:
df['len_chars'] = df[texto_col].str.len() # columna de longitud de caracteres
df['len_tokens_raw'] = df[texto_col].str.split().str.len() # columna de tokens sin procesar
df[['len_chars','len_tokens_raw']].describe()

,len_chars,len_tokens_raw
count,30414.000000,30414.000000
mean,69.090123,10.828270
std,69.670394,10.993367
min,1.000000,1.000000
25%,15.000000,2.000000
50%,47.000000,7.000000
75%,102.000000,16.000000
max,300.000000,61.000000


#### Eliminación de nulos y duplicados


In [7]:
before = len(df)
df = df.dropna(subset=[texto_col])
df = df.drop_duplicates(subset=[texto_col])
print(f"Filas eliminadas: {before - len(df)}")

Filas eliminadas: 11088


### **05. Selección de Concepto**


In [8]:
# Distribución de frecuencias para elegir uno con suficientes observaciones
concepto_col = "concepto_nombre" 
concept_counts = df[concepto_col].value_counts()
concept_counts.head(20)

concepto_nombre
Otro                                                      3976
Igualdad                                                   519
Democracia                                                 507
Descentralización                                          503
Justicia                                                   492
Respeto/conservación de la naturaleza o medio ambiente     484
A la vida                                                  440
A la salud                                                 437
A la educación                                             428
Respeto                                                    385
Fuerzas Armadas                                            381
Bien Común/comunidad                                       381
Congreso o parlamento (estructura y funciones)             373
A la vivienda digna                                        365
Plebiscitos, referendos y consultas                        361
Respeto por la constitución            

#### Selección de Concepto

In [9]:
concepto_elegido = "Democracia" 

#### Creación del Subconjunto filtrado
Se elige el concepto 'Democracia' el cual tiene una frecuencia de (507 ocurrencias) para garantizar masa textual suficiente y diversidad léxica. Se descartaron conceptos con baja frecuencia para evitar resultados inestables en nubes de palabras y bigramas.”



In [10]:
subset = df[df[concepto_col] == concepto_elegido].copy()
print("Filas del concepto:", len(subset))


Filas del concepto: 507


In [69]:
# Subconjunto de datos para el concepto elegido
subset.head(10)

,region_nombre,provincia_nombre,comuna_nombre,cabildo_id,cabildo_nombre,cabildo_direccion,num_mesa,pregunta_glosa,concepto_nombre,concepto_otro,tipo-eleccion,fundamento,len_chars,len_tokens_raw
1,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Sin fundamento,14,2
142,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",5,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Que el ciudadano pueda participar en la toma d...,141,22
169,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",6,¿Cuáles son los valores y principios más impor...,Democracia,NaN,P,Las condiciones actuales no constituyen democr...,118,17
283,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",9,¿Cuáles son los valores y principios más impor...,Democracia,NaN,P,por la estructura establecida en el dialogo an...,52,8
316,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",10,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,debemos pasar de una democracia representativa...,66,9
363,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",11,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,por eso estamos aqui,20,4
491,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",14,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Se garantice una democracia vinculante que se ...,89,14
526,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",15,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,la base y el principio de todos,31,7
746,Arica y Parinacota,Parinacota,Putre,121,Liceo Granaderos de Putre,"Teniente del Campo 242, Putre",2,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Mejorar en todos los aspectos los sistemas de...,258,41
945,Tarapacá,Iquique,Alto Hospicio,107,San Antonio de Matilla,"Los Kiwis 3443, Alto Hospicio",2,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Elemento fundamental en la nueva constitución ...,54,8


### **06. Pre-Procesamiento de Texto**


#### Normalización de Texto

In [14]:
def normalize_text(text: str) -> str:
    text = str(text).lower() #
    text = unicodedata.normalize('NFKD', text).encode('ascii','ignore').decode('utf-8','ignore')  # eliminar tildes
    text = re.sub(r'[^a-z0-9\\s]', ' ', text) # eliminar caracteres especiales
    text = re.sub(r'\\s+', ' ', text).strip() # eliminar espacios extra
    return text

subset['texto_norm'] = subset[texto_col].apply(normalize_text)

#### Tokenización + POS + Lematización

In [15]:
def stanza_process(doc_str: str):
    doc = nlp(doc_str)
    rows = []  # (token, lemma, upos)
    for sent in doc.sentences:
        for w in sent.words:
            rows.append((w.text, w.lemma.lower(), w.upos))
    return rows

subset['doc'] = subset['texto_norm'].progress_apply(stanza_process)

100%|██████████| 507/507 [01:33<00:00,  5.44it/s]
